In [ ]:
from tkinter import *
from PIL import ImageTk, Image
import itertools   
import serial
import time
import pandas as pd
import numpy as np
import sys
import glob
import serial

In [ ]:
def create_CSV():
    PORTS = serial_ports()
    for port_name in PORTS:
        OUTPUTS =[]
        
        x_vals = []
        y_vals = []
        if(port_name[9] == 'u'):
            ser = serial.Serial(port = port_name, baudrate = 9600, timeout = 0)
            with serial.Serial(port_name, baudrate=9600) as ser:
                for i in range(100):
                    s = ser.read(4)        # read up to ten bytes (timeout)
                    OUTPUTS.append(int.from_bytes(s[:-1], "little"))
        df = pd.DataFrame(OUTPUTS, columns=[port_name])
    df.to_csv('~/Desktop/PORTS.csv', index=False)
    info_clicked("Succesfully Created the CSV file", info_frame)

In [ ]:
def info_clicked(value, info_frame):
    for widget in info_frame.winfo_children():
        widget.destroy()
    myLabel = Label(info_frame, text = value)
    myLabel.place(in_ = info_frame, x = 0, y= -20)

In [ ]:
def serial_ports():
    """ Lists serial port names

        :raises EnvironmentError:
            On unsupported or unknown platforms
        :returns:
            A list of the serial ports available on the system
    """
    if sys.platform.startswith('win'):
        ports = ['COM%s' % (i + 1) for i in range(256)]
    elif sys.platform.startswith('linux') or sys.platform.startswith('cygwin'):
        # this excludes your current terminal "/dev/tty"
        ports = glob.glob('/dev/tty[A-Za-z]*')
    elif sys.platform.startswith('darwin'):
        ports = glob.glob('/dev/tty.*')
    else:
        raise EnvironmentError('Unsupported platform')

    result = []
    for port in ports:
        try:
            s = serial.Serial(port)
            s.close()
            result.append(port)
        except (OSError, serial.SerialException):
            pass
    return result

In [ ]:
def serial_port_processing(port_list):
    new_list = []
    for i in range(1, len(port_list)+1):
        new_list.append(("Port N" + str(i), port_list[i-1]))
    return (new_list)

In [ ]:
def test_run():
    data = pd.read_csv('/Users/armenmkrtumyan/Desktop/oqni/Exp#1.csv')
    data_top = Toplevel()
    data_top.geometry('800x1000+700+0')
    data_top.title("Test Run")
    
    for i in range(len(data)):
        frame = Frame(data_top, width=600, height=400)
        frame.pack()
        frame.place(anchor='center', relx=0.5, rely=0.5)
        leg_image = PhotoImage(file='/Users/armenmkrtumyan/Desktop/oqni/' \
                               + str(data['label_index'][i]) +'.png')
        my_label = Label(frame, image=leg_image)
        my_label['image'] = leg_image
        my_label.pack()
        info_clicked(data['labels'][i], info_frame)
        data_top.update()
        time.sleep(data['time_interval'][i])
        for widget in frame.winfo_children():
            widget.destroy()
    info_clicked('', info_frame)
    data_top.destroy()

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg

def open_graph(port_name):
    plt.xlim(-5, 5)
    plt.ylim([0, 400])
    top = Toplevel()
    top.title("Graph")
    top.geometry('+%d+%d'%(0,490))  

    #top.geometry('700x490+0+490')
    
    figure = plt.Figure(figsize=(9.3,6.6), dpi=100)    
    
    x = pd.read_csv('PORTS.csv')[port_name]
    y = np.arange(1,len(x)+1)

    df = pd.DataFrame(list(zip(x, y)))
    ax = figure.add_subplot(111)
    ax.set_ymargin(2)
    ax.set_xmargin(0)
    ax.set_title('Graph of the port ' + port_name)
    
    
    chart_type = FigureCanvasTkAgg(figure, top)
    chart_type.get_tk_widget().pack()
    df = df.groupby(1).sum()

    df.plot(kind='line', legend=True, ax=ax)


In [ ]:
root = Tk()
root.title("Port Reader")
root.geometry('700x400+0+0')
root.minsize(700, 400)
root.maxsize(700, 400)

bg = PhotoImage(file='/Users/armenmkrtumyan/Downloads/abstract-science-background-molecule-abstract-science-background-molecule-cells-beautiful-glow-10081.png')
my_label = Label(root, image=bg)
my_label.place(x=0,y=0, relwidth=1, relheight=1)

port_frame = LabelFrame(root, text="PORTS", padx = 0, pady = 0)
port_frame.place(x=10,y=10,height=380,width=100)
actions_frame = LabelFrame(root, text="ACTIONS", padx = 50, pady = 50)
actions_frame.place(x=120, y = 10, height = 160, width = 200)
info_frame = LabelFrame(root, text="Information", padx = 50, pady = 50)
info_frame.place(x = 330, y = 10, height = 100, width = 300)

PORTS = (serial_port_processing(serial_ports()))

port_variable = StringVar()


for text, mode in PORTS:
    Radiobutton(port_frame, text = text, variable=port_variable, value=mode).pack()

graph_button = Button(actions_frame, text = "Open graph", command = lambda: open_graph(port_variable.get()))
graph_button.place(in_ = actions_frame, x = -40, y= -40)
info_button = Button(actions_frame, text="Info about port", command= lambda: info_clicked(port_variable.get(), info_frame))
info_button.place(in_ = actions_frame, x = -40, y= -10)
CSV_button = Button(actions_frame, text = "Create CSV", command = create_CSV)
CSV_button.place(in_ = actions_frame, x = -40, y= 20)
pre_made_data_button = Button(actions_frame, text = "Run the test", command = lambda: test_run())
pre_made_data_button.place(in_ = actions_frame, x = -40, y= 50)

myLabel = Label(info_frame, text = "Select a label to see the output")
myLabel.place(in_ = info_frame, x = 0, y= -20)

mainloop()